# Mission - Classifiez automatiquement des informations

Vous êtes mandaté en tant que Consultant Data Scientist par le département RH de votre client. Il s’agit de l'ESN TechNova Partners, spécialisée dans le conseil en transformation digitale et la vente d’applications en SaaS.
 
Ils font face à un turnover plus élevé que d'habitude et ils souhaitent identifier les causes racines potentielles derrière ces démissions.

### Importation des librairies

In [1]:
# Librairies "classiques"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
from IPython.display import Image, display

# Librairies scikit-learn
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV, 
    cross_validate,
)
# Preprocess et modèles
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score, precision_recall_curve, roc_auc_score, auc, f1_score, balanced_accuracy_score

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool

# Fonctions python créées
from utils import prepare_xy
from utils import scaler_ou_non

### On reprend les éléments du notebook de l'étape 2

In [2]:
donnees_modelisation = pd.read_csv("../Data/Processed/Projet_4_etape2_clean.csv")

# Partie 5 - Optimisez et interprétez le comportement du modèle

### Recommandation 1 - Reprenez les mêmes principes que le projet 3 (si vous suivez ce parcours dans sa totalité) pour le fine-tuning.
* Afin de trouver les meilleurs hyperparamètres de notre modèle sélectionné, CatBoost, nous utilisons un GridSearchCV pour les rechercher.

#### Récupération de notre séparation X et y et de nos features

In [3]:
X, y = prepare_xy(donnees_modelisation)
features_a_scaler, features_encodees = scaler_ou_non()

#### Recherche des hyperparamètres

In [16]:
# Séparation de X et y
X, y
# Test en jeu d'apprentissage et de test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42, stratify = y)
# Transformation des colonnes et enregistrement dans un pipeline
preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(),features_a_scaler),
        ('cat','passthrough',features_encodees)
    ])
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model',CatBoostClassifier(random_state = 42, verbose=0))
])
# Enregistrement des paramètres
param_grid = {
    'model__iterations' : [200, 300, 500],
    'model__depth' : [2, 3, 4],
    'model__learning_rate' : [0.01, 0.02, 0.03, 0.05],
    'model__l2_leaf_reg': [1, 3, 5, 7, 10],
    'model__class_weights' : [[1, 3], [1, 5], [1, 7]],
}
# Mise en place du GridSearchCV
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    estimator = pipeline,
    param_grid = param_grid,
    cv = cv,
    n_jobs = -1,
    scoring='balanced_accuracy',
    verbose = 0
)
# Entraînement du modèle
grid_search.fit(X_train, y_train)
# J'enregistre quel est le meilleur modèle, je réalise la prédiction et j'évalue
best_model = grid_search.best_estimator_
print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleur balanced accuracy :", grid_search.best_score_)

Meilleurs paramètres : {'model__class_weights': [1, 5], 'model__depth': 3, 'model__iterations': 500, 'model__l2_leaf_reg': 3, 'model__learning_rate': 0.01}
Meilleur balanced accuracy : 0.7486815388670603
